You might need to install this on your system:

apt-get install python3-opencv git

In [1]:
import os
#"""
# !rm k -r
if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1384, done.
remote: Counting objects: 100% (833/833), done.
remote: Compressing objects: 100% (536/536), done.
remote: Total 1384 (delta 612), reused 511 (delta 296), pack-reused 551
Receiving objects: 100% (1384/1384), 15.00 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (968/968), done.
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=57143 sha256=43193df08471b053dccfd06377305eaf9f7a353229924e6dbf8a05f63142ad54
  Stored in directory: /tmp/pip-ephem-wheel-cache-liiif7m5/wheels/c1/8a/57/56dbba25eff58e52e536543

In [2]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
import PIL
import cv2
#print("Pillow version:", PIL.PILLOW_VERSION)
from sklearn.metrics import classification_report

Tensorflow version: 2.7.0
Keras version: 2.7.0


In [8]:
from tensorflow.keras import mixed_precision
#mixed_precision.set_global_policy('mixed_float16')

In [ ]:
if not os.path.isdir('up'):
  !git clone https://github.com/joaopauloschuler/PlantDoc-Object-Detection-Dataset.git up

In [ ]:
up_data_dir = os.getcwd()+"/up/TRAIN"
up_test_dir = os.getcwd()+"/up/TEST"
data_dir = "cropped_train/"
test_dir = "cropped_test/"
all_dir = "cropped_all/"

In [3]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [ ]:
def CropImages(orig_data_dir, dest_data_dir, csv_file, include_original=False, verbose=True, prefix='img_'):
  # Load the CSV file.
  with open (csv_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(',')
      a_lines.append(a_line)
    line_count = line_count + 1
  #a_lines = np.array(a_lines, dtype=object)
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+a_line[3]
    dest_file_name = dest_folder_name+'/'+prefix+str(line_count)+'.jpg'
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    print(line_count,':', a_line, dest_file_name)
    if last_image_file != orig_file_name:
      if os.path.isfile(orig_file_name):
        img = cv2.imread(orig_file_name)
        #cv2_imshow(img)
        last_image_file = orig_file_name
        can_load = True
        original_count = original_count + 1
        if verbose: print("Original imgage shape",img.shape)
        if (include_original):
          if not os.path.isdir(dest_folder_name):
            os.mkdir(dest_folder_name)
          cv2.imwrite(dest_folder_name+'/ori_'+prefix+str(original_count)+'.jpg',img)  
      else:
        if verbose: print('File does not exist:', orig_file_name)
        can_load = False
        #break
    if (can_load):
      y1 = int(a_line[4])
      x1 = int(a_line[5])
      y2 = int(a_line[6])
      x2 = int(a_line[7])

      x_min = np.min([x1, x2])
      x_max = np.max([x1, x2])
      y_min = np.min([y1, y2])
      y_max = np.max([y1, y2])

      crop_img = img[x_min:x_max, y_min:y_max]
      if verbose: print("Cropped imgage shape",crop_img.shape,':', x_min, x_max, y_min, y_max,' deltas:',x_max - x_min, y_max - y_min)
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if ( (crop_img.shape[0]==0) or (crop_img.shape[1]==0)):
        failed_count = failed_count + 1
        if verbose: print("Failed cropping.");
      else:
        cv2.imwrite(dest_file_name,crop_img)
    else:
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(data_dir):
  CropImages(orig_data_dir=up_data_dir, dest_data_dir=data_dir, csv_file='up/train_labels.csv', include_original=False, prefix='train_')

In [ ]:
if not os.path.isdir(test_dir):
  CropImages(orig_data_dir=up_test_dir, dest_data_dir=test_dir, csv_file='up/test_labels.csv', include_original=False, prefix='test_')

In [ ]:
print(os.listdir(data_dir))
print(os.listdir(test_dir))
if not os.path.isdir('cropped_test/Potato leaf'):
  !mkdir 'cropped_test/Potato leaf'
if not os.path.isdir('cropped_test/Tomato two spotted spider mites leaf'):
  !mkdir 'cropped_test/Tomato two spotted spider mites leaf'

['Apple leaf', 'Tomato leaf late blight', 'Apple rust leaf', 'Potato leaf late blight', 'Soyabean leaf', 'Tomato leaf bacterial spot', 'Peach leaf', 'Cherry leaf', 'Tomato Septoria leaf spot', 'Corn rust leaf', 'Tomato leaf yellow virus', 'Tomato Early blight leaf', 'Bell_pepper leaf', 'Corn leaf blight', 'Corn Gray leaf spot', 'Tomato leaf', 'Strawberry leaf', 'Potato leaf early blight', 'Tomato leaf mosaic virus', 'Raspberry leaf', 'Tomato mold leaf', 'grape leaf', 'Tomato two spotted spider mites leaf', 'Blueberry leaf', 'Apple Scab Leaf', 'Potato leaf', 'Squash Powdery mildew leaf', 'grape leaf black rot', 'Bell_pepper leaf spot']
['Apple leaf', 'Tomato leaf late blight', 'Apple rust leaf', 'Potato leaf late blight', 'Soyabean leaf', 'Tomato leaf bacterial spot', 'Peach leaf', 'Cherry leaf', 'Tomato Septoria leaf spot', 'Corn rust leaf', 'Tomato leaf yellow virus', 'Tomato Early blight leaf', 'Bell_pepper leaf', 'Corn leaf blight', 'Corn Gray leaf spot', 'Tomato leaf', 'Strawberry 

In [ ]:
if not os.path.isdir(all_dir):
  !mkdir cropped_all
  ! cp -r cropped_test/* cropped_all/
  ! cp -r cropped_train/* cropped_all/

In [ ]:
datagen = cai.util.create_image_generator(rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.1) 

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [6]:
num_classes = 29
batch_size = 64
epochs = 75
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=all_dir, lab=False, 
  verbose=True, bipolar=False, base_model_name='plant_doc',
  has_training=True, has_validation=True, has_testing=True,
  smart_resize=True,
  training_size=0.65, validation_size=0.15, test_size=0.2,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  29  classes.
smart resize is enabled.
loading train images
train shape is: (5761, 224, 224, 3)
loading validation images
validation shape is: (1333, 224, 224, 3)
loading test images
test shape is: (1789, 224, 224, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0
Loaded.
(5761, 224, 224, 3) (1333, 224, 224, 3) (1789, 224, 224, 3)
(5761, 29) (1333, 29) (1789, 29)


In [4]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 2, 13]:
      basefilename = 'JP30A03-EfficientNet-CroppedPlantDoc-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      
      if kType==-1:
        model = cai.efficientnet.EfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes)
      else:
        model = cai.efficientnet.kEfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes,
          kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)

      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): 
        model.summary()
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=max([multiprocessing.cpu_count(), 4]) # this option 
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [9]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False)

Running: JP30A03-EfficientNet-CroppedPlantDoc--1
Model: "efficientnet-b0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['stem_conv_pad[0][0]']          
                                )                                                                 
                                   

# Fitting

In [ ]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=True)

Running: JP30A03-EfficientNet-CroppedPlantDoc--1
Epoch 1/75
91/91 [==============================] - 65s 484ms/step - loss: 3.5750 - accuracy: 0.1125 - val_loss: 3.1964 - val_accuracy: 0.0953

Epoch 00001: val_accuracy improved from -inf to 0.09527, saving model to JP30A03-EfficientNet-CroppedPlantDoc--1-best_result.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/75
91/91 [==============================] - 42s 446ms/step - loss: 3.1209 - accuracy: 0.1432 - val_loss: 3.2714 - val_accuracy: 0.1125

Epoch 00002: val_accuracy improved from 0.09527 to 0.11253, saving model to JP30A03-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 3/75
91/91 [==============================] - 42s 446ms/step - loss: 3.0288 - accuracy: 0.1623 - val_loss: 4.7306 - val_accuracy: 0.0930

Epoch 00003: val_accuracy did not improve from 0.11253
Epoch 4/75
91/91 [==============================] - 42s 446ms/step - loss: 2.9049 - accuracy: 0.1819 - val_loss: 3.7192 - val_accuracy: 0.1620

Epoch 00004: val_accuracy improved from 0.11253 to 0.16204, saving model to JP30A03-EfficientNet-CroppedPlantDoc--1-best_result.hdf5
Epoch 5/75
91/91 [==============================] - 42s 447ms/step - loss: 2.8192 - accuracy: 0.2060 - val_loss: 4.9038 - val_accuracy: 0.1013

Epoch 00005: val_accuracy did not improve from 0.16204
Epoch 6/75
91/91 [==============================

ResourceExhaustedError: ignored

# Test Results

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True)

# Calculate F1

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True, calc_f1=True)